In [1]:
import findspark

findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, transform

In [2]:
spark = SparkSession.builder.appName("ProductsPrice") \
    .master("yarn") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

23/08/18 07:06:10 WARN Utils: Your hostname, SeongGils-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.219.137 instead (on interface en0)
23/08/18 07:06:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/18 07:06:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/18 07:06:14 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
tranFile = spark.read.text("hdfs://localhost:9000/data/products/ch04_data_transactions.txt")
tranData = tranFile.rdd.map(lambda line: line[0].split("#"))

In [4]:
tranData.count()

1000

In [5]:
from collections import namedtuple
### RDD -> DATE FRAME
from datetime import date, time

transaction = namedtuple("transaction", ["tran_date", "tran_time", "user_id", "product_id", "count", "price"])


def parse_line_transaction(line):
    return transaction(
        line[0].replace("\"", ""),
        line[1].replace("\"", ""),
        line[2].replace("\"", ""),
        line[3].replace("\"", ""),
        line[4].replace("\"", ""),
        line[5].replace("\"", "")
    )

In [6]:
tranData_with_header = tranData.map(lambda line: parse_line_transaction(line))
tranData_with_header.collect()[:10]

[transaction(tran_date='2015-03-30', tran_time='6:55 AM', user_id='51', product_id='68', count='1', price='9506.21'),
 transaction(tran_date='2015-03-30', tran_time='7:39 PM', user_id='99', product_id='86', count='5', price='4107.59'),
 transaction(tran_date='2015-03-30', tran_time='11:57 AM', user_id='79', product_id='58', count='7', price='2987.22'),
 transaction(tran_date='2015-03-30', tran_time='12:46 AM', user_id='51', product_id='50', count='6', price='7501.89'),
 transaction(tran_date='2015-03-30', tran_time='11:39 AM', user_id='86', product_id='24', count='5', price='8370.2'),
 transaction(tran_date='2015-03-30', tran_time='10:35 AM', user_id='63', product_id='19', count='5', price='1023.57'),
 transaction(tran_date='2015-03-30', tran_time='2:30 AM', user_id='23', product_id='77', count='7', price='5892.41'),
 transaction(tran_date='2015-03-30', tran_time='7:41 PM', user_id='49', product_id='58', count='4', price='9298.18'),
 transaction(tran_date='2015-03-30', tran_time='9:18 

In [7]:
transaction_df = spark.createDataFrame(tranData_with_header)
transaction_df.createOrReplaceTempView("transaction")

spark.sql("SELECT * FROM transaction limit 5").show()


+----------+---------+-------+----------+-----+-------+
| tran_date|tran_time|user_id|product_id|count|  price|
+----------+---------+-------+----------+-----+-------+
|2015-03-30|  6:55 AM|     51|        68|    1|9506.21|
|2015-03-30|  7:39 PM|     99|        86|    5|4107.59|
|2015-03-30| 11:57 AM|     79|        58|    7|2987.22|
|2015-03-30| 12:46 AM|     51|        50|    6|7501.89|
|2015-03-30| 11:39 AM|     86|        24|    5| 8370.2|
+----------+---------+-------+----------+-----+-------+


In [10]:
products = spark.read.text("hdfs://localhost:9000/data/products/ch04_data_products.txt").rdd \
    .map(lambda line : line[0].split("#")).map(lambda line : change_str_to_int(line))


def change_str_to_int(line):
    line[0] = int(line[0])

    return line


In [17]:
Product = namedtuple("Product", ["product_id", "product_name", "price", "count"])

def parse_line_product(line):
    return Product(
        int(line[0]),
        str(line[1]),
        float(line[2]),
        int(line[3]),
    )

products_with_header = products.map(lambda line : parse_line_product(line))
products_with_header.collect()[0]

Product(product_id=1, product_name='ROBITUSSIN PEAK COLD NIGHTTIME COLD PLUS FLU', price=9721.89, count=10)

In [19]:
products_df = spark.createDataFrame(products_with_header)
products_df.createOrReplaceTempView("product")
spark.sql("SELECT * FROM product").show()

+----------+--------------------+-------+-----+
|product_id|        product_name|  price|count|
+----------+--------------------+-------+-----+
|         1|ROBITUSSIN PEAK C...|9721.89|   10|
|         2|Mattel Little Mom...|6060.78|    6|
|         3|Cute baby doll, b...|1808.79|    2|
|         4|           Bear doll|  51.06|    6|
|         5|LEGO Legends of C...| 849.36|    6|
|         6|         LEGO Castle|4777.51|   10|
|         7|         LEGO Mixels|8720.91|    1|
|         8|      LEGO Star Wars|7592.44|    4|
|         9|LEGO Lord of the ...| 851.67|    2|
|        10|     LEGO The Hobbit|7314.55|    9|
|        11|      LEGO Minecraft|5646.81|    3|
|        12|   LEGO Hero Factory| 6911.2|    1|
|        13|   LEGO Architecture| 604.58|    5|
|        14|        LEGO Technic|7423.48|    3|
|        15|LEGO Storage & Ac...|3125.96|    2|
|        16|        LEGO Classic| 9933.3|   10|
|        17|   LEGO Galaxy Squad|5593.16|    4|
|        18|     LEGO Mindstorms|6022.88

In [42]:
spark.sql(
    " select "
    "   t2.product_id,"
    "   t2.product_name,"
    "   sum(t1.price) as price_per_product"
    " from transaction as t1"
    " join product as t2 on t1.product_id = t2.product_id"
    " group by t2.product_id, t2.product_name"
).show()

+----------+--------------------+------------------+
|product_id|        product_name| price_per_product|
+----------+--------------------+------------------+
|        26|Barbie Beach Ken ...|60695.880000000005|
|        29|  Intel Core i3 3220| 52398.94999999999|
|        65|Roller Derby Roll...| 58670.56999999999|
|        19|    LEGO Minifigures| 46486.00000000001|
|        54|Essentials Medal ...|20736.739999999998|
|        22| LEGO Speed Champion| 76892.01999999999|
|        77|Treatment Set TS3...|          40571.76|
|         7|         LEGO Mixels|          74330.11|
|        34|GAM X360 Assassin...| 62592.42999999999|
|        50|  LG LED TV 32LN575S|          61318.78|
|        94|ATOPALM MUSCLE AN...|          31049.07|
|        57|Notebook Lenovo U...|          85557.16|
|        32| Intel Core i7 3770K|          34104.55|
|        84|      Cyanocobalamin| 75192.53000000001|
|        31|  Intel Core i5 3570|          75445.77|
|        39|Medal Of Honor Al...|          187

In [28]:
### with out DataFrame

tranByProd = tranData.map(lambda tran : (int(tran[3]), tran))
tranByProd.collect()[0][1]

['2015-03-30', '6:55 AM', '51', '68', '1', '9506.21']

In [32]:
totalsByProd = tranByProd.mapValues(lambda t : float(t[5])).reduceByKey(
    lambda tot1, tot2 : (tot1 + tot2)
)

In [41]:
totalsByProd.filter(lambda column : column[0] == 19).collect()

[(19, 46486.00000000001)]

In [8]:
spark.stop()